In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import muspy
from pathlib import Path
from tqdm.notebook import tqdm
from src.utils import sequence_extraction

# Sequence Extraction with a Single Song

In [2]:
# song = muspy.read('../midi/examples/bitmidi/pink-panther.mid')
song = muspy.read('../data/reference_data/raw/theorytab_midi/a/a-day-to-remember/downfall-of-us-all/intro-and-verse_symbol_key.mid')
print(song)
song[0][-1]

Music(metadata=Metadata(schema_version='0.0', source_filename='intro-and-verse_symbol_key.mid', source_format='midi'), resolution=480, tempos=[Tempo(time=0, qpm=129.00007740004645)], key_signatures=[KeySignature(time=0, root=0, mode='major')], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], tracks=[Track(program=0, is_drum=False, name='melody', notes=[Note(time=7200, pitch=41, duration=120, velocity=80), Note(time=7320, pitch=43, duration=120, velocity=80), Note(time=7440, pitch=47, duration=240, velocity=80), ...])])


Note(time=30480, pitch=62, duration=240, velocity=80)

In [3]:
success = sequence_extraction.extract_melodies_to_folder(song, 'downfall-of-us-all', 'test_set', 4)
print("Sequence extraction returned", success)

Sequence extraction returned 1


## Check if track is long enough for at least one call-and-response pair

In [23]:

end_time = song.tracks[0][-1].time / song.resolution
length_in_bars = 4
quarters_per_bar = song.time_signatures[0].numerator * 4 / song.time_signatures[0].denominator

if round(end_time / quarters_per_bar) >= length_in_bars * 2:
    print("Track is long enough.")

print("End time: %f" % end_time)
print("Duration in bars: " + str(end_time / quarters_per_bar))
print("Required duration in bars: " + str(length_in_bars*2))

End time: 0.015427
Duration in bars: 0.0038568054687500002
Required duration in bars: 8


# Sequence Extraction from a Dataset
Using a Set of crawled Midis from Theorytab

In [4]:
source_folder = '../data/reference_data/raw/theorytab_midi'  
sequence_extraction.split_all_midis_from_folder(source_folder, 'theorytab', 4)

0it [00:00, ?it/s]

18167 songs processed in 561.4556267261505 seconds.
17752 call-and-response pairs created.
1666 songs had no melody track.
4758 songs were too short.
0 songs could not be processed due to an error.
